In [2]:
from deepface import DeepFace
import cv2

def grab_facial_areas(img, detector_backend="opencv", threshold=130, anti_spoofing=True, spoof_threshold=0.8):
    try:
        face_objs = DeepFace.extract_faces(
            img_path=img,
            detector_backend=detector_backend,
            anti_spoofing=anti_spoofing
        )
        faces = []
        for face_obj in face_objs:
            if face_obj["facial_area"]["w"] > threshold:
                is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= spoof_threshold)
                faces.append((
                    face_obj["facial_area"]["x"],
                    face_obj["facial_area"]["y"],
                    face_obj["facial_area"]["w"],
                    face_obj["facial_area"]["h"],
                    is_real,
                    face_obj["antispoof_score"]
                ))
        return faces
    except Exception as e:
        print(f"Error: {e}")
        return []

def main():
    cap = cv2.VideoCapture(0)
    while True:
        _, img = cap.read()
        faces = grab_facial_areas(img, anti_spoofing=True, spoof_threshold=0.8)  # Adjust threshold here (0.8 = 80%)
        if faces:
            for x, y, w, h, is_real, score in faces:
                color = (0, 255, 0) if is_real else (0, 0, 255)
                label = "Real" if is_real else "Spoof"
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, f"{label} - {score:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.imshow("Anti-Spoofing Demo", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or 

In [1]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace
from face_recognition import face_encodings, face_distance

# Configuration
ENCODINGS_FILE = 'data/encodings.pkl'
SPOOF_THRESHOLD = 0.7  # Minimum score to consider face real
RECOGNITION_THRESHOLD = 0.5  # Face recognition tolerance (lower is stricter)
DETECTOR_BACKEND = "opencv"  # or "retinaface", "mtcnn", etc.

def load_encodings():
    """Load facial encodings from PKL file"""
    try:
        with open(ENCODINGS_FILE, 'rb') as f:
            data = pickle.load(f)
        print(f"Loaded {len(data['encodings'])} encodings for {len(set(data['names']))} people")
        return data['encodings'], data['names']
    except Exception as e:
        print(f"Error loading encodings: {e}")
        return [], []

def extract_face_encodings(face_img):
    """Extract 128D face encodings"""
    rgb_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    encodings = face_encodings(rgb_face)
    return encodings[0] if encodings else None

def recognize_face(face_encoding, known_encodings, known_names):
    """Compare face with known encodings"""
    distances = face_distance(known_encodings, face_encoding)
    best_match_idx = np.argmin(distances)
    min_distance = distances[best_match_idx]
    
    if min_distance <= RECOGNITION_THRESHOLD:
        return known_names[best_match_idx], 1 - min_distance
    return None, None

def main():
    # Load known faces
    known_encodings, known_names = load_encodings()
    if not known_encodings:
        print("No encodings found! Please create encodings first.")
        return

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video capture")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Couldn't read frame")
            break

        # Detect faces with anti-spoofing
        try:
            face_objs = DeepFace.extract_faces(
                img_path=frame,
                detector_backend=DETECTOR_BACKEND,
                anti_spoofing=True,
                enforce_detection=False
            )
        except Exception as e:
            print(f"Face detection error: {e}")
            continue

        for face_obj in face_objs:
            x, y, w, h = face_obj["facial_area"]["x"], face_obj["facial_area"]["y"], \
                         face_obj["facial_area"]["w"], face_obj["facial_area"]["h"]
            
            # Check if face is real
            is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= SPOOF_THRESHOLD)
            
            if is_real:
                # Extract face ROI and get encoding
                face_roi = frame[y:y+h, x:x+w]
                face_encoding = extract_face_encodings(face_roi)
                
                if face_encoding is not None:
                    # Recognize face
                    name, confidence = recognize_face(face_encoding, known_encodings, known_names)
                    label = f"{name} ({confidence:.2f})" if name else "Unknown"
                    color = (0, 255, 0)  # Green for recognized
                else:
                    label = "No encoding"
                    color = (0, 255, 255)  # Yellow for detection issues
            else:
                label = f"Spoof ({face_obj['antispoof_score']:.2f})"
                color = (0, 0, 255)  # Red for spoofed

            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        cv2.imshow("Secure Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loaded 574 encodings for 6 people


In [ ]:
from deepface import DeepFace
import cv2

def grab_facial_areas(img, detector_backend="opencv", threshold=130, anti_spoofing=False):
    try:
        face_objs = DeepFace.extract_faces(img_path=img, detector_backend=detector_backend, expand_percentage=0, anti_spoofing=anti_spoofing)
        faces = [
            (
                face_obj["facial_area"]["x"],
                face_obj["facial_area"]["y"],
                face_obj["facial_area"]["w"],
                face_obj["facial_area"]["h"],
                face_obj["is_real"],
                face_obj["antispoof_score"]
            )
            for face_obj in face_objs
            if face_obj["facial_area"]["w"] > threshold
        ]
        return faces
    except Exception as e:
        print(str(e))
        return []

def main():
    cap = cv2.VideoCapture(0)
    while True:
        _, img = cap.read()
        faces = grab_facial_areas(img, anti_spoofing=True)
        if faces:
            print(faces)
            for x, y, w, h, is_real, antispoof_score in faces:
                if is_real:
                    color = (0, 255, 0)
                    real = "Real"
                else:
                    color = (0, 0, 255)
                    real = "Spoof"
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.rectangle(img, (x, y+h), (x+w, y+h+30), color, cv2.FILLED)
                cv2.putText(img, f"{real} - {antispoof_score:.2f}%", (x+5, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        cv2.imshow("frame", img)
        key = cv2.waitKey(1)
        if key == ord("q"):
            break

if __name__ == "__main__":
    main()

[(795, 85, 307, 307, True, 0.9433561861515045), (0, 216, 356, 356, True, 0.8553857207298279)]
[(777, 122, 335, 335, True, 0.6304318904876709), (158, 373, 391, 346, False, 0.5825231969356537)]
[(779, 102, 305, 305, True, 0.527402475476265), (168, 372, 382, 347, True, 0.624309629201889)]
[(798, 94, 290, 290, True, 0.7153775840997696)]
[(782, 97, 303, 303, True, 0.6742955148220062), (156, 377, 380, 342, True, 0.8004303574562073)]
[(133, 334, 392, 385, False, 0.5912452191114426), (793, 104, 333, 333, True, 0.7020304799079895)]
[(777, 107, 317, 317, False, 0.51532331854105), (0, 213, 322, 322, True, 0.9603157937526703)]
[(786, 102, 306, 306, True, 0.6160606741905212)]
[(788, 94, 307, 307, True, 0.6917622834444046)]
[(781, 93, 309, 309, True, 0.8405901491641998)]
[(780, 92, 305, 305, True, 0.965551108121872)]
[(779, 89, 312, 312, True, 0.9442227482795715)]
[(781, 91, 308, 308, True, 0.9450961947441101)]
[(783, 91, 304, 304, True, 0.9221530556678772), (0, 221, 335, 335, True, 0.50389818847179

: 

In [14]:
import cv2
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from deepface import DeepFace
from face_recognition import face_encodings, face_distance
import os

# Configuration
ENCODINGS_FILE = 'C:\pc\Projects\Project\capturing_dataset\data\encodings.pkl'
ATTENDANCE_FILE = 'C:\pc\Projects\Project\capturing_dataset\data\attendance.csv'
SPOOF_THRESHOLD = 0.7  # Minimum score to consider face real
RECOGNITION_THRESHOLD = 0.5  # Face recognition tolerance (lower is stricter)
DETECTOR_BACKEND = "opencv"  # or "retinaface", "mtcnn", etc.
ATTENDANCE_INTERVAL = timedelta(hours=1)  # Time between allowed attendance marks

def load_encodings():
    """Load facial encodings from PKL file"""
    try:
        with open(ENCODINGS_FILE, 'rb') as f:
            data = pickle.load(f)
        print(f"Loaded {len(data['encodings'])} encodings for {len(set(data['names']))} people")
        return data['encodings'], data['names']
    except Exception as e:
        print(f"Error loading encodings: {e}")
        return [], []

def load_attendance():
    """Load attendance records from CSV"""
    if os.path.exists(ATTENDANCE_FILE):
        df = pd.read_csv(ATTENDANCE_FILE, parse_dates=['timestamp'])
        return df
    return pd.DataFrame(columns=['name', 'timestamp', 'status'])

def save_attendance(df):
    """Save attendance records to CSV"""
    df.to_csv(ATTENDANCE_FILE, index=False)

def can_mark_attendance(name, attendance_df):
    """Check if attendance can be marked (not marked in last hour)"""
    if name is None:
        return False
    
    now = datetime.now()
    last_attendance = attendance_df[attendance_df['name'] == name]
    
    if not last_attendance.empty:
        last_time = pd.to_datetime(last_attendance['timestamp'].iloc[-1])
        return now - last_time >= ATTENDANCE_INTERVAL
    return True

def mark_attendance(name, status, attendance_df):
    """Mark attendance in the dataframe"""
    if name is None:
        return attendance_df
    
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    new_entry = pd.DataFrame([[name, now, status]], 
                            columns=['name', 'timestamp', 'status'])
    return pd.concat([attendance_df, new_entry], ignore_index=True)

def extract_face_encodings(face_img):
    """Extract 128D face encodings"""
    rgb_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    encodings = face_encodings(rgb_face)
    return encodings[0] if encodings else None

def recognize_face(face_encoding, known_encodings, known_names):
    """Compare face with known encodings"""
    distances = face_distance(known_encodings, face_encoding)
    best_match_idx = np.argmin(distances)
    min_distance = distances[best_match_idx]
    
    if min_distance <= RECOGNITION_THRESHOLD:
        return known_names[best_match_idx], 1 - min_distance
    return None, None

def main():
    # Load known faces
    known_encodings, known_names = load_encodings()
    if not known_encodings:
        print("No encodings found! Please create encodings first.")
        return

    # Load attendance records
    attendance_df = load_attendance()
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video capture")
        return

    # Track recently recognized faces to avoid spamming attendance
    recently_recognized = {}

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Couldn't read frame")
            break

        # Detect faces with anti-spoofing
        try:
            face_objs = DeepFace.extract_faces(
                img_path=frame,
                detector_backend=DETECTOR_BACKEND,
                anti_spoofing=True,
                enforce_detection=False
            )
        except Exception as e:
            print(f"Face detection error: {e}")
            continue

        for face_obj in face_objs:
            x, y, w, h = face_obj["facial_area"]["x"], face_obj["facial_area"]["y"], \
                         face_obj["facial_area"]["w"], face_obj["facial_area"]["h"]
            
            # Check if face is real
            is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= SPOOF_THRESHOLD)
            
            if is_real:
                # Extract face ROI and get encoding
                face_roi = frame[y:y+h, x:x+w]
                face_encoding = extract_face_encodings(face_roi)
                
                if face_encoding is not None:
                    # Recognize face
                    name, confidence = recognize_face(face_encoding, known_encodings, known_names)
                    label = f"{name} ({confidence:.2f})" if name else "Unknown"
                    color = (0, 255, 0)  # Green for recognized
                    
                    # Check if we should mark attendance
                    if name and name not in recently_recognized:
                        if can_mark_attendance(name, attendance_df):
                            attendance_df = mark_attendance(name, "Present", attendance_df)
                            save_attendance(attendance_df)
                            print(f"Attendance marked for {name}")
                            recently_recognized[name] = datetime.now()
                else:
                    label = "No encoding"
                    color = (0, 255, 255)  # Yellow for detection issues
            else:
                label = f"Spoof ({face_obj['antispoof_score']:.2f})"
                color = (0, 0, 255)  # Red for spoofed
                
                # Mark spoofing attempt in log
                if "spoof_attempt" not in recently_recognized or \
                   (datetime.now() - recently_recognized.get("spoof_attempt", datetime.min)).seconds > 60:
                    attendance_df = mark_attendance("Spoof Attempt", "Spoofed", attendance_df)
                    save_attendance(attendance_df)
                    recently_recognized["spoof_attempt"] = datetime.now()

            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Clean up recently recognized dictionary
        current_time = datetime.now()
        recently_recognized = {k: v for k, v in recently_recognized.items() 
                             if current_time - v < timedelta(minutes=5)}

        cv2.imshow("Secure Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loaded 785 encodings for 9 people


OSError: [Errno 22] Invalid argument: 'C:\\pc\\Projects\\Project\\capturing_dataset\\data\x07ttendance.csv'

GUI

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import cv2
from PIL import Image, ImageTk
import threading
import queue
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from deepface import DeepFace
from face_recognition import face_encodings, face_distance
import os
import time

class AttendanceSystem:
    def __init__(self):
        # Configuration
        self.ENCODINGS_FILE = 'data/encodings.pkl'
        self.ATTENDANCE_FILE = 'data/attendance.csv'
        self.TIMETABLE_FILE = 'data/timetable.csv'
        self.SPOOF_THRESHOLD = 0.7
        self.RECOGNITION_THRESHOLD = 0.5
        self.DETECTOR_BACKEND = "opencv"
        self.ATTENDANCE_INTERVAL = timedelta(hours=1)
        
        # State variables
        self.running = False
        self.camera = None
        self.known_encodings = []
        self.known_names = []
        self.attendance_df = pd.DataFrame()
        self.timetable_df = pd.DataFrame()
        self.recently_recognized = {}
        self.current_subject = None
        self.current_class = None
        
        # Initialize GUI
        self.root = tk.Tk()
        self.root.title("Smart Attendance System")
        self.root.geometry("1200x800")
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Setup styles
        self.setup_styles()
        
        # Build UI
        self.create_widgets()
        
        # Load data
        self.load_data()
        
        # Start camera thread
        self.start_camera()
    
    def setup_styles(self):
        style = ttk.Style()
        style.configure('TFrame', background='#f0f0f0')
        style.configure('TLabel', background='#f0f0f0', font=('Helvetica', 10))
        style.configure('Header.TLabel', font=('Helvetica', 12, 'bold'))
        style.configure('TButton', font=('Helvetica', 10))
        style.configure('Primary.TButton', foreground='white', background='#0078d7')
        style.configure('Success.TButton', foreground='white', background='#4CAF50')
        style.configure('Danger.TButton', foreground='white', background='#f44336')
        style.configure('TCombobox', font=('Helvetica', 10))
        style.configure('Treeview', font=('Helvetica', 10), rowheight=25)
        style.configure('Treeview.Heading', font=('Helvetica', 10, 'bold'))
    
    def create_widgets(self):
        # Main container
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left panel - Camera and controls
        left_panel = ttk.Frame(main_frame, width=600)
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Camera frame
        camera_frame = ttk.LabelFrame(left_panel, text="Live Camera Feed")
        camera_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        self.camera_label = ttk.Label(camera_frame)
        self.camera_label.pack(fill=tk.BOTH, expand=True)
        
        # Controls frame
        controls_frame = ttk.Frame(left_panel)
        controls_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.start_btn = ttk.Button(
            controls_frame, text="Start", style='Success.TButton',
            command=self.start_recognition
        )
        self.start_btn.pack(side=tk.LEFT, padx=5)
        
        self.stop_btn = ttk.Button(
            controls_frame, text="Stop", style='Danger.TButton',
            command=self.stop_recognition, state=tk.DISABLED
        )
        self.stop_btn.pack(side=tk.LEFT, padx=5)
        
        self.export_btn = ttk.Button(
            controls_frame, text="Export Attendance", style='Primary.TButton',
            command=self.export_attendance
        )
        self.export_btn.pack(side=tk.RIGHT, padx=5)
        
        # Status frame
        status_frame = ttk.LabelFrame(left_panel, text="System Status")
        status_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.status_label = ttk.Label(status_frame, text="System ready. Click Start to begin recognition.")
        self.status_label.pack(fill=tk.X, padx=5, pady=5)
        
        # Right panel - Attendance and timetable
        right_panel = ttk.Frame(main_frame, width=400)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Timetable controls
        timetable_control_frame = ttk.LabelFrame(right_panel, text="Timetable Configuration")
        timetable_control_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Label(timetable_control_frame, text="Current Class:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=2)
        self.class_combo = ttk.Combobox(timetable_control_frame, state="readonly")
        self.class_combo.grid(row=0, column=1, sticky=tk.EW, padx=5, pady=2)
        self.class_combo.bind("<<ComboboxSelected>>", self.update_subject_options)
        
        ttk.Label(timetable_control_frame, text="Current Subject:").grid(row=1, column=0, sticky=tk.W, padx=5, pady=2)
        self.subject_combo = ttk.Combobox(timetable_control_frame, state="readonly")
        self.subject_combo.grid(row=1, column=1, sticky=tk.EW, padx=5, pady=2)
        self.subject_combo.bind("<<ComboboxSelected>>", self.update_current_session)
        
        # Attendance summary
        summary_frame = ttk.LabelFrame(right_panel, text="Today's Attendance Summary")
        summary_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.summary_label = ttk.Label(summary_frame, text="No attendance recorded today")
        self.summary_label.pack(fill=tk.X, padx=5, pady=5)
        
        # Attendance details
        details_frame = ttk.LabelFrame(right_panel, text="Attendance Details")
        details_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Treeview for attendance records
        self.tree = ttk.Treeview(details_frame, columns=('name', 'timestamp', 'status', 'subject'), show='headings')
        self.tree.heading('name', text='Name')
        self.tree.heading('timestamp', text='Time')
        self.tree.heading('status', text='Status')
        self.tree.heading('subject', text='Subject')
        self.tree.column('name', width=120)
        self.tree.column('timestamp', width=120)
        self.tree.column('status', width=80)
        self.tree.column('subject', width=100)
        
        scrollbar = ttk.Scrollbar(details_frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.tree.pack(fill=tk.BOTH, expand=True)
        
        # Configure grid weights
        timetable_control_frame.columnconfigure(1, weight=1)
    
    def load_data(self):
        """Load all required data files"""
        try:
            # Load face encodings
            if os.path.exists(self.ENCODINGS_FILE):
                with open(self.ENCODINGS_FILE, 'rb') as f:
                    data = pickle.load(f)
                self.known_encodings = data['encodings']
                self.known_names = data['names']
                self.update_status(f"Loaded {len(self.known_encodings)} encodings for {len(set(self.known_names))} people")
            else:
                self.update_status("Warning: No face encodings found!", "warning")
            
            # Load attendance records
            if os.path.exists(self.ATTENDANCE_FILE):
                self.attendance_df = pd.read_csv(self.ATTENDANCE_FILE, parse_dates=['timestamp'])
                self.update_attendance_display()
            else:
                self.attendance_df = pd.DataFrame(columns=['name', 'timestamp', 'status', 'subject'])
            
            # Load timetable
            if os.path.exists(self.TIMETABLE_FILE):
                self.timetable_df = pd.read_csv(self.TIMETABLE_FILE)
                self.update_timetable_display()
            else:
                self.timetable_df = pd.DataFrame(columns=['class', 'subject', 'day', 'time'])
                self.update_status("Warning: No timetable found!", "warning")
        except Exception as e:
            self.update_status(f"Error loading data: {str(e)}", "error")
    
    def update_timetable_display(self):
        """Update the timetable dropdowns"""
        if not self.timetable_df.empty:
            classes = sorted(self.timetable_df['class'].unique())
            self.class_combo['values'] = classes
            if classes:
                self.class_combo.current(0)
                self.update_subject_options()
    
    def update_subject_options(self, event=None):
        """Update subject dropdown based on selected class"""
        selected_class = self.class_combo.get()
        if selected_class and not self.timetable_df.empty:
            subjects = self.timetable_df[self.timetable_df['class'] == selected_class]['subject'].unique()
            self.subject_combo['values'] = subjects
            if subjects:
                self.subject_combo.current(0)
                self.update_current_session()
    
    def update_current_session(self, event=None):
        """Update current class and subject"""
        self.current_class = self.class_combo.get()
        self.current_subject = self.subject_combo.get()
        self.update_status(f"Current session: {self.current_class} - {self.current_subject}")
    
    def start_camera(self):
        """Initialize camera in a separate thread"""
        self.camera = cv2.VideoCapture(0)
        if not self.camera.isOpened():
            self.update_status("Error: Could not open video capture", "error")
            return
        
        self.camera_thread = threading.Thread(target=self._camera_loop, daemon=True)
        self.camera_thread.start()
    
    def _camera_loop(self):
        """Thread for continuously capturing camera frames"""
        while self.running or not hasattr(self, 'camera_queue'):
            ret, frame = self.camera.read()
            if ret:
                if not hasattr(self, 'camera_queue'):
                    self.camera_queue = queue.Queue(maxsize=1)
                
                try:
                    # Convert to RGB and resize for display
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame_resized = cv2.resize(frame_rgb, (640, 480))
                    
                    # Put frame in queue (discard old frame if queue is full)
                    if self.camera_queue.full():
                        self.camera_queue.get_nowait()
                    self.camera_queue.put(frame_resized)
                except queue.Full:
                    pass
            time.sleep(0.03)  # ~30 FPS
    
    def update_camera_display(self):
        """Update the camera display in the GUI"""
        if hasattr(self, 'camera_queue') and not self.camera_queue.empty():
            frame = self.camera_queue.get()
            
            # Convert to PhotoImage
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            
            # Update label
            self.camera_label.imgtk = imgtk
            self.camera_label.configure(image=imgtk)
        
        # Schedule next update
        if self.running:
            self.root.after(30, self.update_camera_display)
    
    def start_recognition(self):
        """Start face recognition process"""
        if not self.known_encodings:
            messagebox.showerror("Error", "No face encodings loaded!")
            return
        
        if not self.current_subject or not self.current_class:
            messagebox.showwarning("Warning", "Please select class and subject first!")
            return
        
        self.running = True
        self.start_btn.config(state=tk.DISABLED)
        self.stop_btn.config(state=tk.NORMAL)
        
        # Start processing thread
        self.processing_thread = threading.Thread(target=self._processing_loop, daemon=True)
        self.processing_thread.start()
        
        # Start camera display updates
        self.update_camera_display()
        self.update_status("Face recognition started")
    
    def stop_recognition(self):
        """Stop face recognition process"""
        self.running = False
        self.start_btn.config(state=tk.NORMAL)
        self.stop_btn.config(state=tk.DISABLED)
        self.update_status("Face recognition stopped")
    
    def _processing_loop(self):
        """Thread for face recognition processing"""
        while self.running:
            if hasattr(self, 'camera_queue') and not self.camera_queue.empty():
                frame = self.camera_queue.get()
                
                # Convert back to BGR for processing
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                
                try:
                    # Detect faces with anti-spoofing
                    face_objs = DeepFace.extract_faces(
                        img_path=frame_bgr,
                        detector_backend=self.DETECTOR_BACKEND,
                        anti_spoofing=True,
                        enforce_detection=False
                    )
                    
                    for face_obj in face_objs:
                        x, y, w, h = face_obj["facial_area"]["x"], face_obj["facial_area"]["y"], \
                                     face_obj["facial_area"]["w"], face_obj["facial_area"]["h"]
                        
                        # Check if face is real
                        is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= self.SPOOF_THRESHOLD)
                        
                        if is_real:
                            # Extract face ROI and get encoding
                            face_roi = frame_bgr[y:y+h, x:x+w]
                            face_encoding = self.extract_face_encodings(face_roi)
                            
                            if face_encoding is not None:
                                # Recognize face
                                name, confidence = self.recognize_face(face_encoding)
                                label = f"{name} ({confidence:.2f})" if name else "Unknown"
                                color = (0, 255, 0)  # Green for recognized
                                
                                # Check if we should mark attendance
                                if name and name not in self.recently_recognized:
                                    if self.can_mark_attendance(name):
                                        self.mark_attendance(name, "Present")
                                        self.recently_recognized[name] = datetime.now()
                                        self.root.after(0, self.update_attendance_display)
                        else:
                            label = f"Spoof ({face_obj['antispoof_score']:.2f})"
                            color = (0, 0, 255)  # Red for spoofed
                            
                            # Mark spoofing attempt in log
                            if "spoof_attempt" not in self.recently_recognized or \
                               (datetime.now() - self.recently_recognized.get("spoof_attempt", datetime.min)).seconds > 60:
                                self.mark_attendance("Spoof Attempt", "Spoofed")
                                self.recently_recognized["spoof_attempt"] = datetime.now()
                                self.root.after(0, self.update_attendance_display)
                        
                        # Draw on frame (for next display update)
                        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                    
                    # Clean up recently recognized dictionary
                    current_time = datetime.now()
                    self.recently_recognized = {k: v for k, v in self.recently_recognized.items() 
                                             if current_time - v < timedelta(minutes=5)}
                
                except Exception as e:
                    print(f"Processing error: {e}")
                
                time.sleep(0.1)  # Reduce CPU usage
    
    def extract_face_encodings(self, face_img):
        """Extract 128D face encodings"""
        try:
            rgb_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
            encodings = face_encodings(rgb_face)
            return encodings[0] if encodings else None
        except Exception as e:
            print(f"Encoding error: {e}")
            return None
    
    def recognize_face(self, face_encoding):
        """Compare face with known encodings"""
        try:
            distances = face_distance(self.known_encodings, face_encoding)
            best_match_idx = np.argmin(distances)
            min_distance = distances[best_match_idx]
            
            if min_distance <= self.RECOGNITION_THRESHOLD:
                return self.known_names[best_match_idx], 1 - min_distance
            return None, None
        except Exception as e:
            print(f"Recognition error: {e}")
            return None, None
    
    def can_mark_attendance(self, name):
        """Check if attendance can be marked (not marked in last hour)"""
        if not hasattr(self, 'attendance_df'):
            return False
        
        now = datetime.now()
        last_attendance = self.attendance_df[self.attendance_df['name'] == name]
        
        if not last_attendance.empty:
            last_time = pd.to_datetime(last_attendance['timestamp'].iloc[-1])
            return now - last_time >= self.ATTENDANCE_INTERVAL
        return True
    
    def mark_attendance(self, name, status):
        """Mark attendance in the dataframe"""
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        new_entry = pd.DataFrame([[name, now, status, self.current_subject]], 
                               columns=['name', 'timestamp', 'status', 'subject'])
        self.attendance_df = pd.concat([self.attendance_df, new_entry], ignore_index=True)
        
        # Save to file
        try:
            self.attendance_df.to_csv(self.ATTENDANCE_FILE, index=False)
        except Exception as e:
            self.update_status(f"Error saving attendance: {str(e)}", "error")
    
    def update_attendance_display(self):
        """Update the attendance display in the GUI"""
        if not hasattr(self, 'attendance_df'):
            return
        
        # Update treeview
        self.tree.delete(*self.tree.get_children())
        
        # Show today's records
        today = datetime.now().date()
        today_records = self.attendance_df[
            pd.to_datetime(self.attendance_df['timestamp']).dt.date == today
        ]
        
        for _, row in today_records.iterrows():
            self.tree.insert('', tk.END, values=(
                row['name'],
                row['timestamp'],
                row['status'],
                row.get('subject', 'N/A')
            ))
        
        # Update summary
        present_count = len(today_records[today_records['status'] == 'Present'])
        unique_students = today_records[today_records['status'] == 'Present']['name'].nunique()
        self.summary_label.config(
            text=f"Today's attendance: {present_count} records\n{unique_students} unique students"
        )
    
    def export_attendance(self):
        """Export attendance data to file"""
        if self.attendance_df.empty:
            messagebox.showwarning("Warning", "No attendance data to export!")
            return
        
        file_path = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx"), ("All files", "*.*")],
            title="Save attendance data"
        )
        
        if file_path:
            try:
                if file_path.endswith('.xlsx'):
                    self.attendance_df.to_excel(file_path, index=False)
                else:
                    self.attendance_df.to_csv(file_path, index=False)
                self.update_status(f"Attendance data exported to {file_path}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to export data: {str(e)}")
    
    def update_status(self, message, level="info"):
        """Update the status label with a message"""
        colors = {
            "info": "black",
            "warning": "orange",
            "error": "red",
            "success": "green"
        }
        self.status_label.config(text=message, foreground=colors.get(level, "black"))
        print(f"[{level.upper()}] {message}")
    
    def on_close(self):
        """Clean up before closing"""
        self.running = False
        if hasattr(self, 'camera') and self.camera is not None:
            self.camera.release()
        self.root.destroy()
    
    def run(self):
        """Run the application"""
        self.root.mainloop()

if __name__ == "__main__":
    app = AttendanceSystem()
    app.run()

[INFO] Loaded 687 encodings for 8 people
[ERROR] Error loading data: unconverted data remains when parsing with format "%Y-%m-%d %H:%M:%S": ".356669", at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from PIL import Image, ImageTk
import cv2
import threading
import queue
import pandas as pd
from datetime import datetime, timedelta
from deepface import DeepFace
from face_recognition import face_encodings, face_distance
import os
import time
import numpy as np
from tkcalendar import Calendar

# 🌈 Color Theme
BG_COLOR = "#2E3440"  # Dark slate
PRIMARY_COLOR = "#5E81AC"  # Soft blue
SECONDARY_COLOR = "#88C0D0"  # Light blue
ACCENT_COLOR = "#A3BE8C"  # Green
ERROR_COLOR = "#BF616A"  # Red
TEXT_COLOR = "#ECEFF4"  # White

class SmartAttendanceSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("🎤 Smart Attendance System")
        self.root.geometry("1280x800")
        self.root.configure(bg=BG_COLOR)
        
        # 🎭 Face Recognition Config
        self.ENCODINGS_FILE = 'data/encodings.pkl'
        self.ATTENDANCE_FILE = 'data/attendance.csv'
        self.TIMETABLE_FILE = 'data/timetable.csv'
        self.SPOOF_THRESHOLD = 0.7
        self.RECOGNITION_THRESHOLD = 0.5
        self.DETECTOR_BACKEND = "opencv"
        
        # 🚦 System State
        self.running = False
        self.camera = None
        self.known_encodings = []
        self.known_names = []
        self.attendance_df = pd.DataFrame()
        self.timetable_df = pd.DataFrame()
        self.current_subject = None
        self.current_class = None
        self.recently_recognized = {}
        
        # 🎥 Camera Setup
        self.camera_queue = queue.Queue(maxsize=1)
        self.setup_camera()
        
        # 🖼️ GUI Setup
        self.setup_gui()
        
        # 📂 Load Data
        self.load_data()
        
        # ⏳ Auto-Detect Current Class/Subject
        self.update_current_session()
        
    def setup_camera(self):
        """Initialize camera in a separate thread"""
        self.camera = cv2.VideoCapture(0)
        if not self.camera.isOpened():
            messagebox.showerror("Error", "Camera not detected!")
            return
        
        self.camera_thread = threading.Thread(target=self._camera_loop, daemon=True)
        self.camera_thread.start()
    
    def _camera_loop(self):
        """Thread for capturing frames"""
        while True:
            ret, frame = self.camera.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_resized = cv2.resize(frame_rgb, (640, 480))
                
                if self.camera_queue.full():
                    self.camera_queue.get_nowait()
                self.camera_queue.put(frame_resized)
            time.sleep(0.03)  # ~30 FPS
    
    def setup_gui(self):
        """Build the modern GUI"""
        # 🔹 Main Frame
        main_frame = tk.Frame(self.root, bg=BG_COLOR)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # 🎥 Left Panel (Camera + Controls)
        left_panel = tk.Frame(main_frame, bg=BG_COLOR)
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # 📷 Camera Frame
        camera_frame = tk.LabelFrame(left_panel, text="🎥 Live Camera", font=("Helvetica", 12, "bold"), bg=BG_COLOR, fg=TEXT_COLOR)
        camera_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        self.camera_label = tk.Label(camera_frame, bg="black")
        self.camera_label.pack(fill=tk.BOTH, expand=True)
        
        # 🎛️ Controls Frame
        controls_frame = tk.Frame(left_panel, bg=BG_COLOR)
        controls_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.start_btn = tk.Button(
            controls_frame, text="▶️ START", font=("Helvetica", 10, "bold"),
            bg=ACCENT_COLOR, fg="white", relief=tk.FLAT,
            command=self.start_recognition
        )
        self.start_btn.pack(side=tk.LEFT, padx=5)
        
        self.stop_btn = tk.Button(
            controls_frame, text="⏹️ STOP", font=("Helvetica", 10, "bold"),
            bg=ERROR_COLOR, fg="white", relief=tk.FLAT,
            command=self.stop_recognition, state=tk.DISABLED
        )
        self.stop_btn.pack(side=tk.LEFT, padx=5)
        
        self.export_btn = tk.Button(
            controls_frame, text="📤 EXPORT", font=("Helvetica", 10, "bold"),
            bg=PRIMARY_COLOR, fg="white", relief=tk.FLAT,
            command=self.export_attendance
        )
        self.export_btn.pack(side=tk.RIGHT, padx=5)
        
        # 📊 Right Panel (Attendance Dashboard)
        right_panel = tk.Frame(main_frame, bg=BG_COLOR, width=400)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH)
        
        # 🏫 Current Session Info
        session_frame = tk.LabelFrame(right_panel, text="📅 Current Session", font=("Helvetica", 12, "bold"), bg=BG_COLOR, fg=TEXT_COLOR)
        session_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.class_label = tk.Label(session_frame, text="Class: N/A", font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.class_label.pack(anchor=tk.W, padx=5, pady=2)
        
        self.subject_label = tk.Label(session_frame, text="Subject: N/A", font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.subject_label.pack(anchor=tk.W, padx=5, pady=2)
        
        self.time_label = tk.Label(session_frame, text="Time: N/A", font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.time_label.pack(anchor=tk.W, padx=5, pady=2)
        
        # 📈 Attendance Summary
        summary_frame = tk.LabelFrame(right_panel, text="📊 Today's Stats", font=("Helvetica", 12, "bold"), bg=BG_COLOR, fg=TEXT_COLOR)
        summary_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.summary_label = tk.Label(summary_frame, text="No attendance today", font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.summary_label.pack(fill=tk.X, padx=5, pady=5)
        
        # 📝 Attendance Logs
        log_frame = tk.LabelFrame(right_panel, text="📝 Attendance Logs", font=("Helvetica", 12, "bold"), bg=BG_COLOR, fg=TEXT_COLOR)
        log_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # 📜 Treeview (Attendance Table)
        style = ttk.Style()
        style.configure("Treeview", background=BG_COLOR, foreground=TEXT_COLOR, fieldbackground=BG_COLOR)
        style.map("Treeview", background=[('selected', PRIMARY_COLOR)])
        
        self.tree = ttk.Treeview(log_frame, columns=("Name", "Time", "Status", "Subject"), show="headings")
        self.tree.heading("Name", text="Name")
        self.tree.heading("Time", text="Time")
        self.tree.heading("Status", text="Status")
        self.tree.heading("Subject", text="Subject")
        
        scrollbar = ttk.Scrollbar(log_frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.tree.pack(fill=tk.BOTH, expand=True)
        
        # ⏱️ Update Clock
        self.update_clock()
    
    def update_clock(self):
        """Live clock in GUI"""
        now = datetime.now().strftime("%H:%M:%S")
        self.time_label.config(text=f"Time: {now}")
        self.root.after(1000, self.update_clock)
    
    def load_data(self):
        """Load encodings, timetable, and attendance"""
        try:
            # 👥 Load face encodings
            if os.path.exists(self.ENCODINGS_FILE):
                with open(self.ENCODINGS_FILE, 'rb') as f:
                    data = pickle.load(f)
                self.known_encodings = data['encodings']
                self.known_names = data['names']
            
            # 📅 Load timetable
            if os.path.exists(self.TIMETABLE_FILE):
                self.timetable_df = pd.read_csv(self.TIMETABLE_FILE)
            
            # 📝 Load attendance
            if os.path.exists(self.ATTENDANCE_FILE):
                self.attendance_df = pd.read_csv(self.ATTENDANCE_FILE, parse_dates=['timestamp'])
            else:
                self.attendance_df = pd.DataFrame(columns=['name', 'timestamp', 'status', 'subject'])
            
            self.update_attendance_display()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load data: {str(e)}")
    
    def update_current_session(self):
        """Auto-detect current class/subject based on time"""
        if self.timetable_df.empty:
            return
        
        now = datetime.now()
        current_time = now.strftime("%H:%M")
        current_day = now.strftime("%A")
        
        # Filter timetable for current day and time
        active_sessions = self.timetable_df[
            (self.timetable_df['day'] == current_day) &
            (self.timetable_df['time'].str.contains(current_time))
        ]
        
        if not active_sessions.empty:
            self.current_class = active_sessions.iloc[0]['class']
            self.current_subject = active_sessions.iloc[0]['subject']
            
            self.class_label.config(text=f"Class: {self.current_class}")
            self.subject_label.config(text=f"Subject: {self.current_subject}")
    
    def start_recognition(self):
        """Start face recognition"""
        if not self.known_encodings:
            messagebox.showerror("Error", "No face data loaded!")
            return
        
        self.running = True
        self.start_btn.config(state=tk.DISABLED)
        self.stop_btn.config(state=tk.NORMAL)
        
        # Start processing thread
        self.processing_thread = threading.Thread(target=self._process_faces, daemon=True)
        self.processing_thread.start()
        
        # Start camera updates
        self.update_camera_display()
    
    def stop_recognition(self):
        """Stop face recognition"""
        self.running = False
        self.start_btn.config(state=tk.NORMAL)
        self.stop_btn.config(state=tk.DISABLED)
    
    def _process_faces(self):
        """Face recognition thread"""
        while self.running:
            if not self.camera_queue.empty():
                frame = self.camera_queue.get()
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                
                try:
                    # Detect faces with anti-spoofing
                    face_objs = DeepFace.extract_faces(
                        img_path=frame_bgr,
                        detector_backend=self.DETECTOR_BACKEND,
                        anti_spoofing=True,
                        enforce_detection=False
                    )
                    
                    for face_obj in face_objs:
                        x, y, w, h = face_obj["facial_area"]["x"], face_obj["facial_area"]["y"], \
                                     face_obj["facial_area"]["w"], face_obj["facial_area"]["h"]
                        
                        # Check if face is real
                        is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= self.SPOOF_THRESHOLD)
                        
                        if is_real:
                            # Extract face encoding
                            face_roi = frame_bgr[y:y+h, x:x+w]
                            face_encoding = self._extract_encoding(face_roi)
                            
                            if face_encoding is not None:
                                # Recognize face
                                name, confidence = self._recognize_face(face_encoding)
                                
                                # Mark attendance if new detection
                                if name and name not in self.recently_recognized:
                                    self.mark_attendance(name, "Present")
                                    self.recently_recognized[name] = datetime.now()
                                    self.root.after(0, self.update_attendance_display)
                except Exception as e:
                    print(f"Face processing error: {e}")
    
    def mark_attendance(self, name, status):
        """Log attendance with timestamp"""
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_entry = pd.DataFrame([[name, now, status, self.current_subject]], 
                               columns=['name', 'timestamp', 'status', 'subject'])
        self.attendance_df = pd.concat([self.attendance_df, new_entry], ignore_index=True)
        self.attendance_df.to_csv(self.ATTENDANCE_FILE, index=False)
    
    def update_attendance_display(self):
        """Update the attendance dashboard"""
        self.tree.delete(*self.tree.get_children())
        
        today = datetime.now().date()
        today_records = self.attendance_df[
            pd.to_datetime(self.attendance_df['timestamp']).dt.date == today
        ]
        
        for _, row in today_records.iterrows():
            self.tree.insert("", tk.END, values=(
                row['name'],
                row['timestamp'],
                row['status'],
                row['subject']
            ))
        
        # Update summary
        present_count = len(today_records[today_records['status'] == 'Present'])
        unique_students = today_records[today_records['status'] == 'Present']['name'].nunique()
        self.summary_label.config(
            text=f"📌 Today's Attendance\n✅ Present: {unique_students} students\n📋 Total Records: {present_count}"
        )
    
    def update_camera_display(self):
        """Update camera feed in GUI"""
        if not self.camera_queue.empty():
            frame = self.camera_queue.get()
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            
            self.camera_label.imgtk = imgtk
            self.camera_label.configure(image=imgtk)
        
        if self.running:
            self.root.after(30, self.update_camera_display)
    
    def export_attendance(self):
        """Export attendance to CSV/Excel"""
        if self.attendance_df.empty:
            messagebox.showwarning("Warning", "No attendance data to export!")
            return
        
        file_path = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV", "*.csv"), ("Excel", "*.xlsx")],
            title="Save Attendance Report"
        )
        
        if file_path:
            try:
                if file_path.endswith(".xlsx"):
                    self.attendance_df.to_excel(file_path, index=False)
                else:
                    self.attendance_df.to_csv(file_path, index=False)
                messagebox.showinfo("Success", f"Attendance exported to:\n{file_path}")
            except Exception as e:
                messagebox.showerror("Error", f"Export failed: {str(e)}")
    
    def on_close(self):
        """Cleanup before closing"""
        self.running = False
        if self.camera:
            self.camera.release()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = SmartAttendanceSystem(root)
    root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from PIL import Image, ImageTk
import cv2
import threading
import queue
import pandas as pd
from datetime import datetime
import os
import numpy as np
import pickle
from deepface import DeepFace
from concurrent.futures import ThreadPoolExecutor

# 🌈 Color Theme
BG_COLOR = "#2E3440"  # Dark slate
PRIMARY_COLOR = "#5E81AC"  # Soft blue
SECONDARY_COLOR = "#88C0D0"  # Light blue
ACCENT_COLOR = "#A3BE8C"  # Green
ERROR_COLOR = "#BF616A"  # Red
TEXT_COLOR = "#ECEFF4"  # White

class SmartAttendanceSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("🎤 Smart Attendance System")
        self.root.geometry("1280x800")
        self.root.configure(bg=BG_COLOR)
        
        # 🎭 Face Recognition Config
        self.ENCODINGS_FILE = 'data/encodings.pkl'
        self.ATTENDANCE_FILE = 'data/attendance.csv'
        self.TIMETABLE_FILE = 'data/timetable.csv'
        self.SPOOF_THRESHOLD = 0.7
        self.RECOGNITION_THRESHOLD = 0.6
        self.DETECTOR_BACKEND = "opencv"
        self.MIN_FACE_SIZE = 100  # Minimum face size in pixels
        
        # 🚦 System State
        self.running = False
        self.camera = None
        self.known_encodings = []
        self.known_names = []
        self.attendance_df = pd.DataFrame()
        self.timetable_df = pd.DataFrame()
        self.current_subject = "General"
        self.current_class = "Default"
        self.recently_recognized = {}
        self.last_processed_time = 0
        self.processing_interval = 1.0  # Process frames every 1 second
        
        # 🎥 Camera Setup
        self.camera_queue = queue.Queue(maxsize=1)
        self.processed_frame_queue = queue.Queue(maxsize=1)
        self.setup_camera()
        
        # 🧵 Thread Management
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.face_detection_active = False
        
        # 🖼️ GUI Setup
        self.setup_gui()
        
        # 📂 Load Data
        self.load_data()
        
        # ⏳ Auto-Detect Current Class/Subject
        self.update_current_session()
        
        # 🖥️ Start camera display
        self.update_camera_display()
        
        # 🚨 Cleanup on window close
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def setup_camera(self):
        """Initialize camera with optimized settings"""
        self.camera = cv2.VideoCapture(0)
        if not self.camera.isOpened():
            messagebox.showerror("Error", "Camera not detected!")
            return
        
        # Set camera resolution to 640x480 for better performance
        self.camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        # Start camera thread
        self.camera_thread = threading.Thread(target=self._camera_loop, daemon=True)
        self.camera_thread.start()
    
    def _camera_loop(self):
        """Thread for capturing frames with optimized frame skipping"""
        while True:
            ret, frame = self.camera.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Put frame in queue for display
                if self.camera_queue.full():
                    self.camera_queue.get_nowait()
                self.camera_queue.put(frame_rgb)
                
                # Put frame in processing queue if recognition is active
                if self.running and not self.face_detection_active:
                    self.face_detection_active = True
                    self.executor.submit(self._process_frame, frame_rgb.copy())
            
            # Control frame rate
            time.sleep(0.03)  # ~30 FPS
    
    def _process_frame(self, frame):
        """Process a single frame for face detection and recognition"""
        try:
            current_time = time.time()
            
            # Skip processing if not enough time has passed
            if current_time - self.last_processed_time < self.processing_interval:
                self.face_detection_active = False
                return
            
            self.last_processed_time = current_time
            
            # Detect faces with anti-spoofing
            face_objs = DeepFace.extract_faces(
                img_path=frame,
                detector_backend=self.DETECTOR_BACKEND,
                anti_spoofing=True,
                enforce_detection=False
            )
            
            processed_frame = frame.copy()
            
            for face_obj in face_objs:
                x, y, w, h = face_obj["facial_area"]["x"], face_obj["facial_area"]["y"], \
                             face_obj["facial_area"]["w"], face_obj["facial_area"]["h"]
                
                # Skip small faces
                if w < self.MIN_FACE_SIZE or h < self.MIN_FACE_SIZE:
                    continue
                
                # Check if face is real
                is_real = face_obj.get("is_real", True) and (face_obj.get("antispoof_score", 1.0) >= self.SPOOF_THRESHOLD)
                
                if is_real:
                    # Extract face ROI
                    face_roi = frame[y:y+h, x:x+w]
                    
                    # Recognize face
                    name, confidence = self._recognize_face(face_roi)
                    
                    # Draw rectangle and label
                    color = (0, 255, 0) if name else (0, 0, 255)
                    cv2.rectangle(processed_frame, (x, y), (x+w, y+h), color, 2)
                    
                    if name:
                        label = f"{name} ({confidence:.2f})"
                        cv2.putText(processed_frame, label, (x, y-10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                        
                        # Mark attendance if new detection
                        if name not in self.recently_recognized:
                            self.mark_attendance(name, "Present")
                            self.recently_recognized[name] = datetime.now()
                            self.root.after(0, self.update_attendance_display)
            
            # Put processed frame in queue for display
            if self.processed_frame_queue.full():
                self.processed_frame_queue.get_nowait()
            self.processed_frame_queue.put(processed_frame)
            
        except Exception as e:
            print(f"Face processing error: {e}")
        finally:
            self.face_detection_active = False
    
    def _recognize_face(self, face_img):
        """Recognize face using DeepFace with fallback to encodings"""
        try:
            # First try DeepFace recognition
            result = DeepFace.find(
                img_path=face_img,
                db_path="data/faces",
                enforce_detection=False,
                silent=True
            )
            
            if result and len(result) > 0 and len(result[0]) > 0:
                best_match = result[0].iloc[0]
                if best_match['distance'] < self.RECOGNITION_THRESHOLD:
                    name = os.path.basename(os.path.dirname(best_match['identity']))
                    return name, 1 - best_match['distance']
            
            # Fallback to encodings if DeepFace fails
            if self.known_encodings:
                face_encoding = DeepFace.represent(
                    img_path=face_img,
                    model_name="Facenet",
                    enforce_detection=False
                )
                
                if face_encoding:
                    face_encoding = np.array(face_encoding)
                    distances = np.linalg.norm(self.known_encodings - face_encoding, axis=1)
                    min_idx = np.argmin(distances)
                    min_distance = distances[min_idx]
                    
                    if min_distance < self.RECOGNITION_THRESHOLD:
                        return self.known_names[min_idx], 1 - min_distance
            
            return None, 0.0
            
        except Exception as e:
            print(f"Recognition error: {e}")
            return None, 0.0
    
    def setup_gui(self):
        """Build the modern GUI"""
        # 🔹 Main Frame
        main_frame = tk.Frame(self.root, bg=BG_COLOR)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # 🎥 Left Panel (Camera + Controls)
        left_panel = tk.Frame(main_frame, bg=BG_COLOR)
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # 📷 Camera Frame
        camera_frame = tk.LabelFrame(left_panel, text="🎥 Live Camera", font=("Helvetica", 12, "bold"), 
                                   bg=BG_COLOR, fg=TEXT_COLOR)
        camera_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        self.camera_label = tk.Label(camera_frame, bg="black")
        self.camera_label.pack(fill=tk.BOTH, expand=True)
        
        # 🎛️ Controls Frame
        controls_frame = tk.Frame(left_panel, bg=BG_COLOR)
        controls_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.start_btn = tk.Button(
            controls_frame, text="▶️ START RECOGNITION", font=("Helvetica", 10, "bold"),
            bg=ACCENT_COLOR, fg="white", relief=tk.FLAT,
            command=self.start_recognition
        )
        self.start_btn.pack(side=tk.LEFT, padx=5)
        
        self.stop_btn = tk.Button(
            controls_frame, text="⏹️ STOP RECOGNITION", font=("Helvetica", 10, "bold"),
            bg=ERROR_COLOR, fg="white", relief=tk.FLAT,
            command=self.stop_recognition, state=tk.DISABLED
        )
        self.stop_btn.pack(side=tk.LEFT, padx=5)
        
        self.export_btn = tk.Button(
            controls_frame, text="📤 EXPORT ATTENDANCE", font=("Helvetica", 10, "bold"),
            bg=PRIMARY_COLOR, fg="white", relief=tk.FLAT,
            command=self.export_attendance
        )
        self.export_btn.pack(side=tk.RIGHT, padx=5)
        
        # 📊 Right Panel (Attendance Dashboard)
        right_panel = tk.Frame(main_frame, bg=BG_COLOR, width=400)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH)
        
        # 🏫 Current Session Info
        session_frame = tk.LabelFrame(right_panel, text="📅 Current Session", 
                                     font=("Helvetica", 12, "bold"), bg=BG_COLOR, fg=TEXT_COLOR)
        session_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.class_label = tk.Label(session_frame, text="Class: Default", 
                                  font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.class_label.pack(anchor=tk.W, padx=5, pady=2)
        
        self.subject_label = tk.Label(session_frame, text="Subject: General", 
                                    font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.subject_label.pack(anchor=tk.W, padx=5, pady=2)
        
        self.time_label = tk.Label(session_frame, text="Time: --:--:--", 
                                 font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.time_label.pack(anchor=tk.W, padx=5, pady=2)
        
        # 📈 Attendance Summary
        summary_frame = tk.LabelFrame(right_panel, text="📊 Today's Stats", 
                                    font=("Helvetica", 12, "bold"), bg=BG_COLOR, fg=TEXT_COLOR)
        summary_frame.pack(fill=tk.X, padx=5, pady=5)
        
        self.summary_label = tk.Label(summary_frame, text="No attendance today", 
                                    font=("Helvetica", 10), bg=BG_COLOR, fg=TEXT_COLOR)
        self.summary_label.pack(fill=tk.X, padx=5, pady=5)
        
        # 📝 Attendance Logs
        log_frame = tk.LabelFrame(right_panel, text="📝 Attendance Logs", 
                                font=("Helvetica", 12, "bold"), bg=BG_COLOR, fg=TEXT_COLOR)
        log_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # 📜 Treeview (Attendance Table)
        style = ttk.Style()
        style.configure("Treeview", background=BG_COLOR, foreground=TEXT_COLOR, fieldbackground=BG_COLOR)
        style.map("Treeview", background=[('selected', PRIMARY_COLOR)])
        
        self.tree = ttk.Treeview(log_frame, columns=("Name", "Time", "Status", "Subject"), show="headings")
        self.tree.heading("Name", text="Name")
        self.tree.heading("Time", text="Time")
        self.tree.heading("Status", text="Status")
        self.tree.heading("Subject", text="Subject")
        
        scrollbar = ttk.Scrollbar(log_frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.tree.pack(fill=tk.BOTH, expand=True)
        
        # ⏱️ Update Clock
        self.update_clock()
    
    def update_clock(self):
        """Live clock in GUI"""
        now = datetime.now().strftime("%H:%M:%S")
        self.time_label.config(text=f"Time: {now}")
        self.root.after(1000, self.update_clock)
    
    def load_data(self):
        """Load encodings, timetable, and attendance"""
        try:
            # 👥 Load face encodings
            if os.path.exists(self.ENCODINGS_FILE):
                with open(self.ENCODINGS_FILE, 'rb') as f:
                    data = pickle.load(f)
                self.known_encodings = data['encodings']
                self.known_names = data['names']
            
            # 📅 Load timetable
            if os.path.exists(self.TIMETABLE_FILE):
                self.timetable_df = pd.read_csv(self.TIMETABLE_FILE)
            
            # 📝 Load attendance
            if os.path.exists(self.ATTENDANCE_FILE):
                self.attendance_df = pd.read_csv(self.ATTENDANCE_FILE)
            else:
                self.attendance_df = pd.DataFrame(columns=['name', 'timestamp', 'status', 'subject'])
            
            self.update_attendance_display()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load data: {str(e)}")
    
    def update_current_session(self):
        """Auto-detect current class/subject based on time"""
        if self.timetable_df.empty:
            return
        
        now = datetime.now()
        current_time = now.strftime("%H:%M")
        current_day = now.strftime("%A")
        
        # Filter timetable for current day and time
        active_sessions = self.timetable_df[
            (self.timetable_df['day'] == current_day) &
            (self.timetable_df['start_time'] <= current_time) &
            (self.timetable_df['end_time'] >= current_time)
        ]
        
        if not active_sessions.empty:
            self.current_class = active_sessions.iloc[0]['class']
            self.current_subject = active_sessions.iloc[0]['subject']
            
            self.class_label.config(text=f"Class: {self.current_class}")
            self.subject_label.config(text=f"Subject: {self.current_subject}")
    
    def start_recognition(self):
        """Start face recognition"""
        if not self.known_encodings and not os.path.exists("data/faces"):
            messagebox.showerror("Error", "No face data loaded! Please add faces to 'data/faces' folder.")
            return
        
        self.running = True
        self.start_btn.config(state=tk.DISABLED)
        self.stop_btn.config(state=tk.NORMAL)
    
    def stop_recognition(self):
        """Stop face recognition"""
        self.running = False
        self.start_btn.config(state=tk.NORMAL)
        self.stop_btn.config(state=tk.DISABLED)
    
    def mark_attendance(self, name, status):
        """Log attendance with timestamp"""
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_entry = pd.DataFrame([[name, now, status, self.current_subject]], 
                               columns=['name', 'timestamp', 'status', 'subject'])
        self.attendance_df = pd.concat([self.attendance_df, new_entry], ignore_index=True)
        self.attendance_df.to_csv(self.ATTENDANCE_FILE, index=False)
    
    def update_attendance_display(self):
        """Update the attendance dashboard"""
        self.tree.delete(*self.tree.get_children())
        
        today = datetime.now().date()
        today_records = self.attendance_df[
            pd.to_datetime(self.attendance_df['timestamp']).dt.date == today
        ].sort_values('timestamp', ascending=False)
        
        for _, row in today_records.iterrows():
            self.tree.insert("", tk.END, values=(
                row['name'],
                row['timestamp'],
                row['status'],
                row['subject']
            ))
        
        # Update summary
        present_count = len(today_records[today_records['status'] == 'Present'])
        unique_students = today_records[today_records['status'] == 'Present']['name'].nunique()
        self.summary_label.config(
            text=f"📌 Today's Attendance\n✅ Present: {unique_students} students\n📋 Total Records: {present_count}"
        )
    
    def update_camera_display(self):
        """Update camera feed in GUI"""
        # Prioritize showing processed frames if available
        if not self.processed_frame_queue.empty():
            frame = self.processed_frame_queue.get()
        elif not self.camera_queue.empty():
            frame = self.camera_queue.get()
        else:
            frame = None
        
        if frame is not None:
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            
            self.camera_label.imgtk = imgtk
            self.camera_label.configure(image=imgtk)
        
        self.root.after(30, self.update_camera_display)
    
    def export_attendance(self):
        
        """Export attendance to CSV/Excel"""
        if self.attendance_df.empty:
            messagebox.showwarning("Warning", "No attendance data to export!")
            return

    # Ask the user where to save the file
    file_path = filedialog.asksaveasfilename(
        defaultextension=".csv",
        filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")],
        title="Save attendance file"
    )
    
    if not file_path:
        return  # User cancelled

    try:
        if file_path.endswith(".csv"):
            self.attendance_df.to_csv(file_path, index=False)
        elif file_path.endswith(".xlsx"):
            self.attendance_df.to_excel(file_path, index=False)
        else:
            messagebox.showerror("Error", "Unsupported file type selected!")
            return

        messagebox.showinfo("Success", f"Attendance exported successfully to:\n{file_path}")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to export attendance:\n{e}")
    
    def on_close(self):
        """Cleanup before closing"""
        self.running = False
        if self.camera:
            self.camera.release()
        self.executor.shutdown(wait=False)
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = SmartAttendanceSystem(root)
    root.mainloop()

Exception in thread Thread-3:
Traceback (most recent call last):
  File "c:\Users\suryansh\anaconda3\envs\face_recognition_env\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\suryansh\anaconda3\envs\face_recognition_env\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\suryansh\anaconda3\envs\face_recognition_env\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\suryansh\AppData\Local\Temp\ipykernel_19468\4264188998.py", line 109, in _camera_loop
NameError: name 'time' is not defined


KeyError: 'start_time'